In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

In [2]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import re

url = "https://www.vlr.gg/stats"
response = requests.get(url)
html = response.text
soup = BeautifulSoup(html, 'html.parser')
table = soup.find('table', class_='wf-table mod-stats mod-scroll')
table.find_all('th')
header = []
for title in table.find_all('th'):
    header.append(title.get_text(strip=True))
header.remove('Player')
header.remove('Agents')
#'Player', 'Team', 'Agents',
header.insert(0, 'Player')
header.insert(1, 'Team')
header.insert(2, 'Agents')
header.insert(3, '3')
header.insert(4, '4')
t_bodys = table.find_all('tbody')[0]
datas = []
for tr_tag in t_bodys.find_all('tr'):
    player_name = tr_tag.find('div', class_='text-of').text.strip()
    datas.append(player_name)
    team_name = tr_tag.find('div', class_='stats-player-country').text.strip()
    datas.append(team_name)
    agent_names = []
    img_src = [img['src'] for img in tr_tag.find_all('img')]
    for src in img_src:
        start_index = src.find('/agents/') + len('/agents/')
        end_index = src.find('.png')
        agent_name = src[start_index:end_index].capitalize()
        agent_names.append(agent_name)
    combined_agents= ' / '.join(agent_names)
    datas.append(combined_agents)
    td_tags = tr_tag.find_all('td')
    for td in td_tags:
        datas.append(td.get_text(strip=True))
num_columns = 24
table_data = [datas[i:i + num_columns] for i in range(0, len(datas), num_columns)]
df = pd.DataFrame(table_data, columns=header)
df_final = df.drop(['3', '4'], axis=1)
df_final

,Player,Team,Agents,Rnd,R,ACS,K:D,KAST,ADR,KPR,...,FDPR,HS%,CL%,CL,KMax,K,D,A,FK,FD
0,florescent,SR,Jett / Raze,225,1.43,283.8,1.73,74%,189.3,1.03,...,0.16,35%,33%,3/9,35,232,134,43,52,35
1,sarahcat,SR,Viper / Fade / Sova,225,1.31,235.8,1.36,82%,153.5,0.84,...,0.09,27%,8%,1/13,20,190,140,103,19,20
2,kaho,SFU.P,Killjoy / Jett / Reyna,242,1.30,259.7,1.50,75%,174.4,0.93,...,0.09,23%,25%,10/40,31,224,149,38,31,22
3,moose,TTR,Raze / Viper / Cypher,411,1.29,255.9,1.52,76%,168.6,0.93,...,0.10,23%,23%,13/56,30,384,252,95,62,40
4,chloric,LV,Sova / Jett / Raze,497,1.27,219.4,1.31,77%,146.5,0.76,...,0.06,25%,15%,10/66,24,378,289,175,41,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
345,TvirusLuke,DRG,Raze / Jett,372,,226.4,0.93,,,0.79,...,,,,,30,293,316,60,0,0
346,Nicc,DRG,Omen / Kayo / Skye,372,,157.5,0.77,,,0.57,...,,,,,26,213,275,146,0,0
347,Life,ASE,Raze / Jett,388,,219.3,1.02,,,0.77,...,,,,,24,298,291,58,0,0
348,hfmi0dzjc9z7,,Skye / Raze / Sova,388,,204.9,0.99,,,0.71,...,,,,,27,274,277,125,0,0
